## IMPORTS


In [2]:
pip install chromadb openai

  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached durationpy-0.9-py3-none-any.whl.metadata (338 bytes)
  Using cached asgiref-3.8.1-py3-none-any.whl.metadata (9.3 kB)
   ---------------------------------------- 0.0/18.8 MB ? eta -:--:--
   -------------- ------------------------- 6.8/18.8 MB 34.9 MB/s eta 0:00:01
   ------------------------------- -------- 14.7/18.8 MB 36.9 MB

In [12]:
import openai
import chromadb
from openai import OpenAI
from chromadb.utils import embedding_functions

## CONFIGURAÇÃO

In [ ]:
open_ai_api_key = "YOUR_OPENAI_API_KEY"
client_ai = OpenAI(api_key=open_ai_api_key)

# CONFIGURAR O EMBEDDING FUNCTION USANDO OPENAI
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=open_ai_api_key,
    model_name="text-embedding-3-small"  # ou "text-embedding-3-large" se preferir
)

## CRIAR OU ABRIR UM BANCO LOCAL DO CHROMA

In [14]:
client = chromadb.Client()
collection = client.get_or_create_collection(name="digital_transformation", embedding_function=embedding_fn)

## CARREGAR E INSERIR O DOCUMENTO NO CHROMADB

In [6]:
# CARREGAR E INSERIR O DOCUMENTO
with open("digital_transformation.txt", "r", encoding="utf-8") as f:
    texto = f.read()

# DIVIDIR EM TRECHOS (simples aqui, pode usar chunking mais avançado com LangChain se quiser)
chunks = [texto[i:i+500] for i in range(0, len(texto), 500)]

# INSERIR NO VECTOR DATABASE
for i, chunk in enumerate(chunks):
    collection.add(
        documents=[chunk],
        ids=[f"chunk_{i}"],
        metadatas=[{"origem": "digital_transformation"}]
    )

print("Documento inserido com sucesso!")

Documento inserido com sucesso!


## CONSULTA SEMÂNTICA

In [ ]:
consulta = "Quais são os principais desafios para implementar a transformação digital em grandes empresas?"
resultados = collection.query(query_texts=[consulta], n_results=5)

for i, r in enumerate(resultados['documents'][0]):
    print(f"\n🔹 Resultado {i+1}:\n{r}")


🔹 Resultado 1:
te orientada por dados é aquela em que todos os níveis organizacionais – do
chão de fábrica ao CEO – entendem o valor dos dados, têm acesso a insights relevantes e confiam
neles para tomar melhores decisões. Construir essa realidade requer liderança, investimento em gente e
paciência, pois cultura não muda da noite para o dia. Mas os resultados compensam: conforme a
organização amadurece, vemos uma transformação no modo de operar. As discussões passam a girar em
torno de evidências objetivas, ex

🔹 Resultado 2:
os. Por exemplo, num projeto de analytics para otimizar a cadeia de suprimentos, deve-se
ter analistas de dados e pessoal de logística trabalhando lado a lado, desde o diagnóstico do problema até
a implementação da solução encontrada. Esse modelo de squads ou células ágeis com foco em dados vem
ganhando tração em transformações digitais, pois garante que a expertise técnica e conhecimento de
domínio caminhem juntos.
A alta liderança também desempenha um papel det

## FORMATAR RESPOSTA FINAL

In [17]:
contexto = "\n\n".join(resultados["documents"][0])

# CHAMADA À API OpenAI (nova versão)
prompt = f"""
Você é um especialista em transformação digital. Com base nos trechos abaixo do e-book criado pelos alunos, responda à pergunta com clareza, profundidade e linguagem executiva.

### Trechos do e-book:
{contexto}

### Pergunta:
{consulta}

### Resposta:
"""

chat_response = client_ai.chat.completions.create(
    model="gpt-4",  # ou "gpt-3.5-turbo"
    messages=[
        {"role": "system", "content": "Você é um consultor executivo especialista em transformação digital."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=500
)

# EXIBIR RESULTADO
print("\n📘 Resposta formatada:")
print(chat_response.choices[0].message.content)


📘 Resposta formatada:
Os principais desafios para implementar uma cultura de dados em grandes empresas incluem:

1. Mudança de mentalidade e cultura: A construção de uma cultura orientada por dados requer uma mudança de mentalidade em todos os níveis da organização. Isso exige liderança, investimento em pessoas e paciência, pois a cultura não se transforma da noite para o dia. 

2. Colaboração inter-funcional: Para otimizar o uso de dados, é crucial que analistas de dados trabalhem em estreita colaboração com outras funções dentro da empresa, como logística, marketing, finanças, etc. A criação de equipes ágeis com foco em dados pode ajudar a garantir que a expertise técnica e o conhecimento do domínio caminhem juntos.

3. Formação de equipes capacitadas: Um desafio significativo é a lacuna de habilidades em dados entre os funcionários. Muitas organizações precisam investir em treinamento para melhorar a literacia de dados em suas equipes e garantir que elas possam usar efetivamente os